In [1]:
import torch
import random
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from MulticoreTSNE import MulticoreTSNE as TSNE

random.seed(42)

In [2]:
# Construct the list of names
words = list()
with open("names.txt", "r") as infile:
    for line in infile:
        words.append(line.strip())
print(len(words))
print(max(len(w) for w in words))

32033
15


In [3]:
# Mapping of characters to/from integers
special = "."
chars = sorted(list(set("".join(words))))
STOI = {s: i + 1 for i, s in enumerate(chars)}
STOI[special] = 0
ITOS = {i: s for s, i in STOI.items()}
vocab_size = len(STOI)

In [4]:
def build_dataset(words, *, special: str = ".", block_size: int = 3) -> tuple[torch.tensor]:
    (X, Y) = (list(), list())

    for word in words:
        context = [0] * block_size
        for char in word + special:
            ix = STOI[char]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(f"Constructed dataset: X: {X.shape}, Y: {Y.shape}")
    return X, Y


# Train, Development, Test split
block_size = 3
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1], block_size=block_size)
Xdev, Ydev = build_dataset(words[n1:n2], block_size=block_size)
Xtest, Ytest = build_dataset(words[n2:], block_size=block_size)

Constructed dataset: X: torch.Size([182625, 3]), Y: torch.Size([182625])
Constructed dataset: X: torch.Size([22655, 3]), Y: torch.Size([22655])
Constructed dataset: X: torch.Size([22866, 3]), Y: torch.Size([22866])


In [5]:
# Utility function for comparing manually generated gradients to those calculated by pytorch


def cmp(s, dt, t):
    exact = torch.all(dt == t.grad).item()
    approx = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f"{s:15s} | exact: {str(exact):5s} | approximate: {str(approx):5s} | maxdiff: {maxdiff}")

In [6]:
n_hidden = 64
n_embedding = 10

gen = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_embedding), generator=gen)

# Layer 1
kaiming_init = (5 / 3) / ((n_embedding * block_size) ** 0.5)
W1 = torch.randn((block_size * n_embedding, n_hidden), generator=gen)
b1 = torch.randn(n_hidden, generator=gen)

# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=gen) * 0.1
b2 = torch.randn(vocab_size, generator=gen) * 0.1

# BatchNorm parameters
bn_gain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bn_bias = torch.randn((1, n_hidden)) * 0.1
bn_momentum = 0.001

parameters = [C, W1, b1, W2, b2, bn_gain, bn_bias]
print(sum(p.nelement() for p in parameters))

for p in parameters:
    p.requires_grad = True

4137


In [7]:
batch_size = 32
n = batch_size  # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=gen)
Xb, Yb = Xtr[ix], Ytr[ix]  # batch X,Y

In [8]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb]  # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1)  # concatenate the vectors

# Linear layer 1
hprebn = embcat @ W1 + b1  # hidden layer pre-activation

# BatchNorm layer
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1 / (n - 1) * (bndiff2).sum(0, keepdim=True)  # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5) ** -0.5
bn_raw = bndiff * bnvar_inv
hpreact = bn_gain * bn_raw + bn_bias

# Non-linearity
h = torch.tanh(hpreact)  # hidden layer

# Linear layer 2
logits = h @ W2 + b2  # output layer

# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes  # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
    p.grad = None

for t in [
    logprobs,
    probs,
    counts,
    counts_sum,
    counts_sum_inv,
    norm_logits,
    logit_maxes,
    logits,
    h,
    hpreact,
    bn_raw,
    bnvar_inv,
    bnvar,
    bndiff2,
    bndiff,
    hprebn,
    bnmeani,
    embcat,
    emb,
]:
    t.retain_grad()
loss.backward()
loss

tensor(3.3565, grad_fn=<NegBackward0>)

In [58]:
# Construct manual backpropagation through the entire network

# dlogprobs - we want to compute the gradient of the loss with respect to all the elements of the dlogprobs tensor
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / n
cmp("logprobs", dlogprobs, logprobs)

# dprobs - gradient of log probs wrt to the loss is
# ∂loss/∂probs = ∂loss/∂logprobs * ∂logprobs/∂probs
# ∂loss/∂probs = dlogprobs * ∂log(x)/∂x = dlogprobs * (1/x)
dprobs = (1.0 / probs) * dlogprobs
cmp("probs", dprobs, probs)

# dcounts_sum_inv - gradient of csi wrt probs is ∂probs/∂a*b where a = counts and b = csi
# ∂probs / ∂a*b = ∂probs/∂a*b * ∂a*b/∂b = Σ a * dprobs
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
cmp("counts_sum_inv", dcounts_sum_inv, counts_sum_inv)

dcounts_sum = (-(counts_sum**-2)) * dcounts_sum_inv
cmp("counts_sum", dcounts_sum, counts_sum)

# Propagate through both branches of the calculation
dcounts = counts_sum_inv * dprobs
dcounts += torch.ones_like(counts) * dcounts_sum
cmp("counts", dcounts, counts)

# counts = norm_logits.exp()
dnorm_logits = (counts) * dcounts
cmp("norm_logits", dnorm_logits, norm_logits)

dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)
cmp("logit_maxes", dlogit_maxes, logit_maxes)

# Backprop through both branches
dlogits = dnorm_logits.clone()
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
cmp("logits", dlogits, logits)

# The backward pass of a matmul is a matmul
# If logits = h @ W + b then:
# ∂L/∂h = ∂L/∂logits @ W^T
# ∂L/∂W = h^T @ ∂L/∂logits
# ∂L/∂b = ∂L/∂logits summed over the column dimension because of the vertical broadcast
dh = dlogits @ W2.T
cmp("h", dh, h)

dW2 = h.T @ dlogits
cmp("W2", dW2, W2)

db2 = dlogits.sum(0)
cmp("b2", db2, b2)

# Backprop through the hyperbolic tangent function
# If a = tanh(z) = ( e^z - e^-z ) / ( e^z + e^-z )
# Then da/dz = 1 - a^2 where a is the output of the tanh function
dhpreact = (1 - h**2) * dh  # local derivative times the chain rule
cmp("hpreact", dhpreact, hpreact)

# Backprop through the batchnorm layer
# Want to backprop into the gain, raw and bias
dbngain = (bn_raw * dhpreact).sum(0, keepdim=True)  # compensate for vertical broadcasting
cmp("bngain", dbngain, bn_gain)

dbnbias = (1.0 * dhpreact).sum(0, keepdim=True)  # compensate for vertical broadcasting
cmp("bnbias", dbnbias, bn_bias)

dbnraw = bn_gain * dhpreact
cmp("bnraw", dbnraw, bn_raw)

dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
cmp("bnvar_inv", dbnvar_inv, bnvar_inv)

dbnvar = (-0.5 * (bnvar + 1e-5) ** (-0.5 - 1)) * dbnvar_inv
cmp("bnvar", dbnvar, bnvar)

dbndiff2 = (1.0 / (n - 1)) * torch.ones_like(bndiff2) * dbnvar
cmp("bndiff2", dbndiff2, bndiff2)

dbndiff = bnvar_inv * dbnraw
dbndiff += (2 * bndiff) * dbndiff2
cmp("bndiff", dbndiff, bndiff)

dbnmeani = (-dbndiff).sum(0)
cmp("bnmeani", dbnmeani, bnmeani)

dhprebn = dbndiff.clone() + (1 / n) * torch.ones_like(hprebn) * dbnmeani
cmp("hprebn", dhprebn, hprebn)

# Backprop through first linear layer
# The backward pass of a matmul is a matmul
# If hprebn = embcat @ W1 + b1 then:
# ∂L/∂embcat = ∂L/∂hprebn @ W1^T
# ∂L/∂W1 = embcat^T @ ∂L/∂hprebn
# ∂L/∂b = ∂L/∂hprebn summed over the column dimension because of the vertical broadcast
dembcat = dhprebn @ W1.T
cmp("embcat", dembcat, embcat)

dW1 = embcat.T @ dhprebn
cmp("W1", dW1, W1)

db1 = dhprebn.sum(0, keepdim=True)
cmp("b1", db1, b1)

demb = dembcat.view(emb.shape)
cmp("emb", demb, emb)

# Route the gradients backwards through the indexing operations, keeping in mind
# that a row could have been used multiple times.
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k, j]
        dC[ix] += demb[k, j]
cmp("C", dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff: